In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

In [43]:
def reberGrammarInner():
    embeddedReberGrammar = [
        [('B', 1)],           # (state 0) =B=>(state 1)
        [('T', 2), ('P', 3)], # (state 1) =T=>(state 2) or =P=>(state 3)
        [('S', 2), ('X', 4)], # (state 2) =S=>(state 2) or =X=>(state 4)
        [('T', 3), ('V', 5)], # (state 3) =T=>(state 3) or =V=>(state 5)
        [('X', 3), ('S', 6)], # (state 4) =X=>(state 3) or =S=>(state 6)
        [('P', 4), ('V', 6)], # (state 5) =P=>(state 4) or =V=>(state 6)
        [('E', None)]         # (state 6) =E=>(terminal)
    ]
    state = 0
    reberString = ''
    while state is not None:
        char, nextState = embeddedReberGrammar[state][np.random.randint(len(embeddedReberGrammar[state]))]
        reberString += char
        state = nextState
    return reberString

def reberGrammarOuter(innerString):
    embeddedReberGrammar = [
        [('B', 1)],           # (state 0) =B=>(state 1)
        [('T', 2), ('P', 3)], # (state 1) =T=>(state 2) or =P=>(state 3)
        [('T', 4)],            # (state 2) =T=>(state 4)
        [('P', 4)],            # (state 3) =P=>(state 4)
        [('E', None)]         # (state 4) =E=>(terminal)
    ]
    state = 0
    reberString = ''
    while state is not None:
        if state == 2 or state == 3:
            reberString += innerString
        char, nextState = embeddedReberGrammar[state][np.random.randint(len(embeddedReberGrammar[state]))]
        reberString += char
        state = nextState
    return reberString

def validateInnerReberGrammar(innerString):
    embeddedReberGrammar = [
        [('B', 1)],           # (state 0) =B=>(state 1)
        [('T', 2), ('P', 3)], # (state 1) =T=>(state 2) or =P=>(state 3)
        [('S', 2), ('X', 4)], # (state 2) =S=>(state 2) or =X=>(state 4)
        [('T', 3), ('V', 5)], # (state 3) =T=>(state 3) or =V=>(state 5)
        [('X', 3), ('S', 6)], # (state 4) =X=>(state 3) or =S=>(state 6)
        [('P', 4), ('V', 6)], # (state 5) =P=>(state 4) or =V=>(state 6)
        [('E', None)]         # (state 6) =E=>(terminal)
    ]
    state = 0
    for char in innerString:
        found = False
        for c, nextState in embeddedReberGrammar[state]:
            if char == c:
                state = nextState
                found = True
                break
        if not found:
            return False
    return True

def vaidateOuterReberGrammar(outerString):
    embeddedReberGrammar = [
        [('B', 1)],           # (state 0) =B=>(state 1)
        [('T', 2), ('P', 3)], # (state 1) =T=>(state 2) or =P=>(state 3)
        [('T', 4)],            # (state 2) =T=>(state 4)
        [('P', 4)],            # (state 3) =P=>(state 4)
        [('E', None)]         # (state 4) =E=>(terminal)
    ]
    state = 0
    for char in outerString:
        found = False
        for c, nextState in embeddedReberGrammar[state]:
            if char == c:
                state = nextState
                found = True
                break
        if not found:
            return False
    return True

def validateReberGrammar(reberString):
    innerReber = reberString[2:-2]
    outerReber = reberString[:2] + reberString[-2:]

    return validateInnerReberGrammar(innerReber) and vaidateOuterReberGrammar(outerReber)

def generateCorruptedGrammar():
    generated_string = reberGrammarOuter(reberGrammarInner())
    # print("Good String:     ", generated_string)
    while validateReberGrammar(generated_string):
        good_list = list(generated_string)
        idx = np.random.randint(len(good_list))
        good_list[idx] = np.random.choice([c for c in 'BTPSXVE'])
        generated_string = ''.join(good_list)
    # print("Corrupted String:", generated_string)
    return generated_string


In [52]:
def generateDataset(size, max_length=30, easier_variant=False):
    # half of the dataset is composed of strings that follow the ERG rules
    string_sequences = [reberGrammarOuter(reberGrammarInner()) for _ in range(size // 2)]
    labels = [1.] * (size // 2)

    if easier_variant:
        standard_deviation = 0
        list_of_lengths = []
        for i in range(100000):
            genStringLen = len(reberGrammarOuter(reberGrammarInner()))
            list_of_lengths.append(genStringLen)

        mean = np.mean(list_of_lengths)
        print(f"Mean Length of ERG String: {mean}")

        for i in range(100000):
            standard_deviation += (list_of_lengths[i] - mean)**2

        standard_deviation = np.sqrt(standard_deviation/100000)
        
        # one eight is composed of random strings
        for _ in range(size // 8):
            length = int(np.random.normal(mean, standard_deviation))
            reberString = ''
            while len(reberString) < length:
                reberString += np.random.choice(['B', 'E', 'T', 'P', 'S', 'X', 'V'])
            string_sequences.append(reberString)
        labels += [0.] * (size // 8)

        # one eight is composed of inner RG strings
        for _ in range(size // 8):
            reberString = reberGrammarInner()
            string_sequences.append(reberString)
        labels += [0.] * (size // 8)

        # one fourth is composed of random strings wrapped in outer RG strings
        for _ in range(size // 4):
            length = int(np.random.normal(mean, standard_deviation)) - 4 # 4 is the length added by the outer RG
            reberString = ''
            while len(reberString) < length:
                reberString += np.random.choice(['B', 'E', 'T', 'P', 'S', 'X', 'V'])
            string_sequences.append(reberGrammarOuter(reberString))
        labels += [0.] * (size // 4)
    else:
        # half of the dataset is composed of corrupted strings
        for _ in range(size // 2):
            string_sequences.append(generateCorruptedGrammar())
        labels += [0.] * (size // 2)

    tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, )
    tokenizer.fit_on_texts(string_sequences)
    string_sequences = tokenizer.texts_to_sequences(string_sequences)
    string_sequences = tf.keras.preprocessing.sequence.pad_sequences(string_sequences, padding='post', maxlen=max_length)

    features = tf.constant(string_sequences)
    labels = tf.constant(labels)

    dataset = tf.data.Dataset.from_tensor_slices((features, labels))

    return dataset, tokenizer

In [60]:
max_length = 30
dataset, tokenizer = generateDataset(10000, max_length)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset = (
    dataset.take(train_size)
    .shuffle(buffer_size=train_size)  # Shuffle before batching
    .batch(32)  # Apply batching
    .cache()  # Cache the dataset for better performance
    .prefetch(tf.data.AUTOTUNE)
)

val_dataset = (
    dataset.skip(train_size)
    .take(val_size)
    .batch(32)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

test_dataset = (
    dataset.skip(train_size + val_size)
    .take(test_size)
    .batch(32)
)

vocab_size = len(['B','E', 'T', 'P', 'S', 'X', 'V']) + 1
embedding_dim = 32

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.GRU(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_dataset, epochs=20, validation_data=val_dataset)

accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy[1]}")

Epoch 1/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.6477 - loss: 0.6525 - val_accuracy: 0.2280 - val_loss: 1.0157
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.7243 - loss: 0.5754 - val_accuracy: 0.3620 - val_loss: 0.8744
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.8212 - loss: 0.4312 - val_accuracy: 0.7160 - val_loss: 0.5558
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.9325 - loss: 0.2050 - val_accuracy: 0.8250 - val_loss: 0.4651
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.9663 - loss: 0.1255 - val_accuracy: 0.9330 - val_loss: 0.2292
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.9625 - loss: 0.1317 - val_accuracy: 0.9620 - val_loss: 0.1068
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.9814 - loss: 0.0736 - val_accuracy: 0.9660 - val_loss: 0.0731
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.9898 - loss: 0.0373 - val_acc

In [75]:
test_strings = ["BPBTSSSSSSSSSSXXTTTTTTTTVPSETE", # incorrect T at the end should be P
                "BPBTSSSSSSSSSSXXTTTTTTTTVPSEPE"] # correct

X_test = tokenizer.texts_to_sequences(test_strings)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding='post', maxlen=max_length)

y_proba = model.predict(X_test)
print()
print("Estimated probability that these are Reber strings:")
for index, string in enumerate(test_strings):
    print("{}: {:.2f}%".format(string, 100 * y_proba[index][0]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

Estimated probability that these are Reber strings:
BPBTSSSSSSSSSSXXTTTTTTTTVPSETE: 8.39%
BPBTSSSSSSSSSSXXTTTTTTTTVPSEPE: 99.99%
